In [1]:
import os
import numpy as np
import logging
import inspect
import re

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, f1_score

from SASentimentModel import SASentimentModel
from kaggle_dataset import KaggleDataSet

from sa_model_pipeline import SAModelPipeline
from sa_model_config_loader import SAModelConfigLoader
from sa_data_loader import SADataLoader
from sa_model_params import SAModelParams
from SASelfAttentionModel import SASelfAttentionModel
from SARnnModel import SARnnModel

C:\Users\dingg\anaconda3\envs\tf_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [3]:
### Extract epoch number from checkpoint file
### example: 'epoch03' will return 3, else 0 if no number is found

def extract_initial_epoch_from_checkpoint(path):
    match = re.search(r"epoch(\d+)", path)
    return int(match.group(1)) if match else 0

In [4]:
def main():

    ### TOGGLE
    
    ### If no existing model to load, set LOAD_EXISTING_MODEL to False (train from scratch)
    ### If there is an existing model to load, set LOAD_EXISTING_MODEL to True and load the CHECKPOINT_PATH
    
    LOAD_EXISTING_MODEL = True
    CHECKPOINT_PATH = "model_checkpoint_epoch02_valacc0.5286.keras"  # adjust to the saved keras model
    
    ### 
    ### To run your model, 
    ###
    ### 1) Change model_module_name to match your model's module name, i.e., the file name
    ### 2) Change model_class_name to match  your model's class name like SASelfAttentionModel
    ### 3) Look for this line at the bottom of the method,
    ###    sa_sentiment_model = SASelfAttentionModel(sa_model_params)
    ###    Change SASelfAttentionModel to match your model's class.
    ###    That's it.
    ### 4) Then run this notebook from start to finish.
    ###


    ###
    ### Modify model_module_name and model_class_name to match your model's 
    ### module and class name and then change the model instance to match your model class below
    ###
    model_module_name = "SARnnModel"
    model_class_name = "SARnnModel"

    ### Load the model configuration and then find the model specified above
    sa_model_config_loader = SAModelConfigLoader()
    list_of_model_config = sa_model_config_loader.load_model_config()
    model_config = sa_model_config_loader.find_model_config(model_module_name, model_class_name)
    
    ###
    ### Construct the path to the train, test and validation CSV file specified in the model config file
    ### for the given model
    ###
    cwd = os.getcwd()        
    train_file = model_config.get_model_train_csv_file_name()
    test_file = model_config.get_model_test_csv_file_name()
    validation_file = model_config.get_model_validation_csv_file_name()

    path_to_train_csv_file = os.path.join(cwd, train_file)
    path_to_test_csv_file = os.path.join(cwd, test_file)
    path_to_validation_csv_file = os.path.join(cwd, validation_file)

    ### Load the train, test and validation data file specific to the model
    sa_data_loader = SADataLoader(path_to_train_csv_file, path_to_test_csv_file, path_to_validation_csv_file)
    sa_data_loader.load_data(KaggleDataSet.get_kaggle_column_names())

    ### Construct the model parameter object
    sa_model_params = SAModelParams(sa_data_loader, model_config)
    logger.info(f"Start running model: {model_module_name}:{model_class_name}") 

    ###
    ### CHANGE THE MODEL TO YOUR MODEL CLASS!!
    ### 

    sa_sentiment_model = SARnnModel(sa_model_params)
    
    ### Register model/preprocessing
    sa_sentiment_model.register(sa_model_params)
    sa_sentiment_model.preprocess(sa_model_params)
    
    ### Checkpoint
    if LOAD_EXISTING_MODEL:
        initial_epoch = extract_initial_epoch_from_checkpoint(CHECKPOINT_PATH) ### epoch number from checkpoint filename
        sa_sentiment_model.load_model_from_file(CHECKPOINT_PATH) ### load saved model weights
        logger.info(f"Resuming training from epoch {initial_epoch}")
        sa_sentiment_model.fit(sa_model_params, resume_training=True, initial_epoch=initial_epoch) ### continue training
        sa_sentiment_model.predict(sa_model_params)
        sa_sentiment_model.evaluate(sa_model_params)
    ### Call the SASentimentModel's run() which will run the model pipeline
    else:
        sa_sentiment_model.run(sa_model_params)
    ###
    ###
    ###


    logger.info(f"Finished running model: {sa_sentiment_model.__class__.__name__}") 


In [5]:
if __name__ == "__main__":
    main()

Transform set to: <bound method BinaryLabelTransformer.transform of <binary_label_transformer.BinaryLabelTransformer object at 0x000001EFFFEC8430>>, Type: <class 'method'>
Path to train csv file: C:\Users\dingg\IE 7500\IE7500-GroupProject-main\train_60K.csv
Path to test csv file: C:\Users\dingg\IE 7500\IE7500-GroupProject-main\test_20K.csv
Path to validation csv file: C:\Users\dingg\IE 7500\IE7500-GroupProject-main\validate_20K.csv
Train size: 60000
Test size: 20000
Validation size: 20000


2025-06-26 18:37:14,863 - INFO - Start running model: SARnnModel:SARnnModel
2025-06-26 18:37:14,864 - INFO - Calling SARnnModel.register(): RNN Sentiment Model/SARnnModel/SARnnModel/train_60K.csv/test_20K.csv/validate_20K.csv/{'vocab_size': 10000, 'sequence_length': 500, 'embedding_dim': 100, 'epoch': 5, 'batch_size': 32}
2025-06-26 18:37:14,864 - INFO - SARnnModel.register(): Completed
2025-06-26 18:37:14,865 - INFO - Calling SARnnModel.preprocess(): RNN Sentiment Model/SARnnModel/SARnnModel/train_60K.csv/test_20K.csv/validate_20K.csv/{'vocab_size': 10000, 'sequence_length': 500, 'embedding_dim': 100, 'epoch': 5, 'batch_size': 32}
2025-06-26 18:37:23,073 - INFO - SARnnModel.preprocess(): Completed
2025-06-26 18:37:23,075 - INFO - Loading model from model_checkpoint_epoch02_valacc0.5286.keras
2025-06-26 18:37:23,298 - INFO - Model loaded and compiled successfully
2025-06-26 18:37:23,299 - INFO - Resuming training from epoch 2
2025-06-26 18:37:23,300 - INFO - Starting fit; resume_traini

Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - accuracy: 0.5085 - loss: 0.6938
Epoch 3: saving model to model_checkpoint_epoch03_valacc0.5134.keras
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 230s 121ms/step - accuracy: 0.5085 - loss: 0.6938 - val_accuracy: 0.5134 - val_loss: 0.6918
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - accuracy: 0.5222 - loss: 0.6919
Epoch 4: saving model to model_checkpoint_epoch04_valacc0.5000.keras
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 234s 125ms/step - accuracy: 0.5222 - loss: 0.6919 - val_accuracy: 0.5000 - val_loss: 0.6928
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 0.5233 - loss: 0.6923
Epoch 5: saving model to model_checkpoint_epoch05_valacc0.5248.keras
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 231s 123ms/step - accuracy: 0.5233 - loss: 0.6923 - val_accuracy: 0.5248 - val_loss: 0.6925


2025-06-26 18:48:58,337 - INFO - Best val_accuracy: 0.5248 at epoch 5
2025-06-26 18:48:58,338 - INFO - SARnnModel.fit(): Model fitted
2025-06-26 18:48:58,339 - INFO - SARnnModel.fit(): Completed
2025-06-26 18:48:58,339 - INFO - Calling SARnnModel.predict(): RNN Sentiment Model/SARnnModel/SARnnModel/train_60K.csv/test_20K.csv/validate_20K.csv/{'vocab_size': 10000, 'sequence_length': 500, 'embedding_dim': 100, 'epoch': 5, 'batch_size': 32}


625/625 ━━━━━━━━━━━━━━━━━━━━ 20s 31ms/step


2025-06-26 18:49:18,469 - INFO - SARnnModel.predict(): Completed
2025-06-26 18:49:18,471 - INFO - Calling SARnnModel.evaluate(): RNN Sentiment Model/SARnnModel/SARnnModel/train_60K.csv/test_20K.csv/validate_20K.csv/{'vocab_size': 10000, 'sequence_length': 500, 'embedding_dim': 100, 'epoch': 5, 'batch_size': 32}


              precision    recall  f1-score   support

           0       0.54      0.34      0.42     10000
           1       0.52      0.71      0.60     10000

    accuracy                           0.52     20000
   macro avg       0.53      0.52      0.51     20000
weighted avg       0.53      0.52      0.51     20000

625/625 ━━━━━━━━━━━━━━━━━━━━ 22s 35ms/step - accuracy: 0.4016 - loss: 0.7133


2025-06-26 18:49:40,219 - INFO - SARnnModel.evaluate(): Accuracy: 0.5248000025749207, Loss: 0.692497730255127
2025-06-26 18:49:40,220 - INFO - SARnnModel.evaluate(): Completed
2025-06-26 18:49:40,221 - INFO - Finished running model: SARnnModel
